In [1]:
import numpy as np
import gym 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt 
import re
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0, 1"


In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu_device = torch.device("cpu")

no_randomness = False
load_weights = False
semi_random = False

class replay_memory(object):
    def __init__(self, size, sd, b):
        self.max_size = size
        self.storage = []
        self.cur_size = 0
        self.batch_size = b
        self.index = 0 
        
    def add(self, s,a,r,ns,d):
        if self.cur_size < self.max_size:
            self.storage.append([{"s":s, "a":a, "r":r, "ns":ns, "d":d}])
            self.cur_size += 1
        else:
            self.storage.pop(0)
            self.storage.append([{"s":s, "a":a, "r":r, "ns":ns, "d":d}])
            
    def sample(self):
        s = []
        a = []
        r = []
        ns = []
        d = []
        for i in range(self.batch_size):
            indx = torch.randint(self.cur_size, size=(1,)).numpy()[0]
            if no_randomness:
                indx = i
            if semi_random:
                indx = self.index
                if self.index < self.cur_size-1 and self.index < self.max_size-1:
                    self.index += 1
                else:
                    self.index = 0 
            s += [self.storage[indx][0]["s"]]
            a += [self.storage[indx][0]["a"]]
            r += [self.storage[indx][0]["r"]]
            ns += [self.storage[indx][0]["ns"]]
            d += [self.storage[indx][0]["d"]]
            
        return {"s":s, "a":a, "r":r, "ns":ns, "d":d}
    

def get_action_(env, epsilon, action):
    if no_randomness:
        epsilon = 0 
    if semi_random:
        x=np.squeeze(action_ts.detach().numpy())
        if np.sum(x) < 10:
            if 2*np.log(x[0]) - 5 > 3*np.log(x[1]) - 7:
                a = 1
            else:
                a = 0 
        else:
            a = action.argmax().detach().numpy()            
    else:
        rnd = torch.rand((1)).numpy()[0]
        if rnd < epsilon:
            a = torch.randint(env.action_space.n, size=(1,)).numpy()[0]
        else:
            a = action.argmax().detach().to(cpu_device).numpy()
        
    return a
    
class exploration(object):
    def __init__(self, max_,min_,num_eps):
        self.epsilon = max_
        self.min_eps = min_
        self.num_eps = num_eps
        self.eps_red = (max_ - min_)/num_eps
    
    def reduce(self):
        if (self.epsilon > self.min_eps):
            self.epsilon -= self.eps_red 
        
        
class DQN(nn.Module):

    def __init__(self, ni, no):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(ni,10)
        self.fc2 = nn.Linear(10,10)
        self.fc3 = nn.Linear(10,no)
        if no_randomness and not load_weights:
            self.fc1.weight.data.fill_(0.2)
            self.fc2.weight.data.fill_(0.2)
            self.fc3.weight.data.fill_(0.2)
            self.fc1.bias.data.fill_(0.0)
            self.fc2.bias.data.fill_(0.0)
            self.fc3.bias.data.fill_(0.0)
        elif load_weights:
            d=np.loadtxt("fc1.weight", delimiter=",")
            self.fc1.weight.data = torch.tensor(d, dtype=torch.float32)
            d=np.loadtxt("fc2.weight", delimiter=",")
            self.fc2.weight.data = torch.tensor(d, dtype=torch.float32)
            d=np.loadtxt("fc3.weight", delimiter=",")
            self.fc3.weight.data = torch.tensor(d, dtype=torch.float32)
            d=np.loadtxt("fc1.bias", delimiter=",")
            self.fc1.bias.data = torch.tensor(d, dtype=torch.float32)
            d=np.loadtxt("fc2.bias", delimiter=",")
            self.fc2.bias.data = torch.tensor(d, dtype=torch.float32)
            d=np.loadtxt("fc3.bias", delimiter=",")            
            self.fc3.bias.data = torch.tensor(d, dtype=torch.float32)

    def forward(self, x):
        x = F.relu(self.fc1(torch.tensor(x).float().unsqueeze(0)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    
def test(period, avg_q, policy_net, env):
    cnt = 0
    for i in range(100):
        state = env.reset()    
        done = False
        while not done:
            action = policy_net(torch.tensor(state).float().to(device))
            action = get_action_(env, 0, action)
            next_state, reward, done, _ = env.step(action)

            state = next_state
            cnt+=1
            state = next_state
    print (period, avg_q, cnt/100)
    return cnt/100
    
def get_weight_norm(net):
    grad_norm=0
    for param in net.parameters():
    #     print(param)
        grad_norm += torch.norm(param)

    return grad_norm

def get_grad_norm(net):
    grad_norm=0
    for param in net.parameters():
    #     print(param)
        grad_norm += torch.norm(param.grad)

    return grad_norm


def get_grad_list(net):
    grads=np.array([])
    for param in net.parameters():
        grads = np.concatenate((grads, param.grad.data.view(-1).detach().numpy()))
    return grads

In [26]:
torch.manual_seed(123)
env = gym.make("CartPole-v0")
# policy_net = DQN(env.observation_space.shape[0], env.action_space.n).to(device) 
model = DQN(env.observation_space.shape[0], env.action_space.n).to(device) 
policy_net = torch.nn.DataParallel(model,device_ids=[0,1],output_device=0).cuda()

target_model = DQN(env.observation_space.shape[0], env.action_space.n).to(device) 
target_net = torch.nn.DataParallel(target_model,device_ids=[0,1],output_device=0).cuda()

optimizer = optim.Adam(policy_net.parameters())
# optimizer = optim.SGD(policy_net.parameters(),lr=0.001)
# torch.nn.utils.clip_grad_norm(policy_net.parameters(),max_norm=10,norm_type=2)

[2019-04-12 09:59:04,673] Making new env: CartPole-v0
/usr/lib/python3.6/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
/usr/lib64/python3.6/site-packages/torch/nn/parallel/data_parallel.py:25: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [27]:
# gym.__file__
# env.reset()
# torch.stack(batch["s"]).shape[1]
# print (policy_net.fc1.weight.data)
# print (policy_net.fc1.bias.data)
# for param in policy_net.parameters():
#     print (param)
    

In [29]:
######################################################################
# Training loop
if semi_random:
    qvtxt = open('Q_value.txt', 'wb')
    tmtxt = open('target_max.txt', 'wb')
    tvtxt = open('target_value.txt', 'wb')
    stxt = open('state_value.txt', 'wb')
    nstxt = open('nstate_value.txt', 'wb')
    atxt = open('action_value.txt', 'wb')
    aqtxt = open('action_q.txt', 'wb')
    bstxt = open('batch_state.txt', 'wb')
    gntxt = open('grad_norm.txt', 'wb')
    wntxt = open('weight_norm.txt', 'wb')
    gtxt = open('grads.txt', 'wb')
    ltxt = open('loss.txt', 'wb')
batch_size = 128
rbm = replay_memory(1000000,env.observation_space, batch_size)
num_episodes = 2000
exp = exploration(0.9, 0.05, num_episodes)
stop_nm = 2080 # stops early at this train step 
show_details = 10060 # will print the details of last xx train_steps
result = []
show_detail = False
target_update = 200 

train_step = 0
for i_episode in range(num_episodes):
     # initialize state
    state = env.reset()
    # Select and perform an action    
    # keep going until get to the goal state
    cnt = 0
    done = False
    while not done:
        cnt+=1
        action_ts = policy_net(torch.tensor(state).float().unsqueeze(0).to(device)).squeeze()
#         action_ts = policy_net(torch.tensor(state).float().to(device))
        action = get_action_(env, exp.epsilon, action_ts)
        next_state, reward, done, _ = env.step(action)
        
        rbm.add(torch.tensor(state), torch.tensor(action), torch.tensor(reward), torch.tensor(next_state), 
               done)
            
        if (no_randomness or train_step > stop_nm - show_details):
            if show_detail:
                print (i_episode,"-",cnt,"-",rbm.cur_size, state, action_ts, action, reward, next_state, done)
            if semi_random:
                np.savetxt(stxt, np.expand_dims(state,0))
                np.savetxt(nstxt, np.expand_dims(next_state,0))
                np.savetxt(atxt, np.expand_dims(action,0))
                np.savetxt(aqtxt, action_ts.detach().numpy())
            
        
        if rbm.cur_size > 100:
            if rbm.cur_size == 101:
                print ("started training")
            batch = rbm.sample()
            
            target_Q = target_net(torch.stack(batch["ns"]).float().to(device
                    )).squeeze().max(2)[0].detach()
            target = 0.99 * target_Q*(1-torch.tensor(batch["d"]).to(device).float()) + \
            torch.stack(batch["r"]).to(device)
            QValue = policy_net(torch.stack(batch["s"]).float().to(
                device)).squeeze().gather(1, torch.stack(batch["a"]).to(device).unsqueeze(1))
            
            loss_model = F.mse_loss(QValue, target.unsqueeze(1))
            loss = torch.nn.DataParallel(loss_model, device_ids=[0,1])

            if (semi_random or no_randomness) and train_step > stop_nm - show_details:
                if show_detail:
                    print (i_episode, "-", train_step)
                    print (np.array(batch["d"],dtype=np.int))
                    print ([i for i in torch.stack(batch["a"]).numpy()])
                tmp = [i for i in target_Q.numpy()]
                if show_detail:
                    print (tmp)
                np.savetxt(tmtxt, np.expand_dims(np.array(tmp),0))
                tmp = [i for i in target.numpy()]
                if show_detail:
                    print (tmp)
                np.savetxt(tvtxt, np.expand_dims(np.array(tmp),0))
                tmp = [i for i in np.squeeze(QValue.detach().numpy())]
                if show_detail:
                    print(tmp)
                np.savetxt(qvtxt, np.expand_dims(np.array(tmp),0))
                np.savetxt(bstxt, np.array(torch.stack(batch["s"]).detach().numpy()))
                            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_step+=1
            
            if (semi_random or no_randomness) and train_step > stop_nm - show_details:
                grad_norm = get_grad_norm(policy_net)
                np.savetxt(gntxt, np.expand_dims(np.array(grad_norm.detach().numpy()),0))    
                grads = get_grad_list(policy_net)
                np.savetxt(gtxt, np.expand_dims(grads,0))    
                weight_norm = get_weight_norm(policy_net)
                np.savetxt(wntxt, np.expand_dims(np.array(weight_norm.detach().numpy()),0))    
                np.savetxt(ltxt, np.expand_dims(np.array(loss.detach().numpy()),0))    
 
                
#             for param in policy_net.parameters():
#                 print(param.grad.data)
                
            if (no_randomness or semi_random) and train_step == stop_nm:
                break
                
        state = next_state
        if train_step > 1 and train_step%target_update==0:
            target_net.load_state_dict(policy_net.state_dict())
#             print ("\n copied wights \n")

            if no_randomness:
                for param in policy_net.parameters():
                    print(param)


    if (no_randomness or semi_random) and train_step == stop_nm:
        break
            
    if train_step>0 and i_episode%100 == 0:
        try:
#             print ("qvalue", QValue)
#             print ("target", target)
            per = test(i_episode, QValue.mean(), policy_net, env)
        except:
            per = test(i_episode, 0, policy_net, env)
            
        result += [ per]
        #     print (cnt)
    exp.reduce()
        
if semi_random:
    env.close()
    qvtxt.close()
    tmtxt.close()
    tvtxt.close()
    stxt.close()
    bstxt.close()
    nstxt.close()
    atxt.close()
    aqtxt.close()
    gntxt.close()
    wntxt.close()
    gtxt.close()
    ltxt.close()

    
    

started training


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:112: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: The size of tensor a (64) must match the size of tensor b (128) at non-singleton dimension 1

In [25]:
torch.tensor(batch["d"]).reshape(2,64).to(device)
torch.reshape(torch.tensor(batch["d"]),())

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]], device='cuda:0',
       dtype=torch.uint8)

In [39]:
policy_net(torch.stack(batch["s"]).float().to(device)).squeeze().shape
# torch.stack(batch["s"]).shape
# .gather(1, torch.stack(batch["a"]).to(device).unsqueeze(1))
# torch.stack(batch["a"])

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:112: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


torch.Size([2, 64, 2])